#Big Data Finals - Word Co-occurnce in pubMed database
####Asutosh Satapathy {asatapat@andrew.cmu.edu}

###Problem Statement
For this project, I will utilize the Pubmed Central (PMC) open access dataset, implement a system to process the PMC dataset with a user-provided term, and generate a JSON file for use in D3-driven visualization. The goal of this final is to produce a visualization of the topics that most commonly co-occur in the pubmed documents with a user provided term or phrase.

###Introduction
PubMed database contains millions of documents related to various articles. Searching through all of the dat manually is a very daunting (sort of impossible task). Hence, I wll be attempting to help explore the database with minimal effort. The basic idea is given a word, find all other words which are co-occuring. There are various ways to approach this problem. 
The approach which I have implemented is as follows: I will be scanning the entire database for this word. I will see which documents contains this word. And after that, I will be selecting the other words based on their co-occurrence in the document.
Again, this can be done in various ways. But before I delve into the nifty details, let's first cover some of the basics of text mining which I learnt during the course of this project. This will be helpful in understanding the underlying concept of the application.
- Tokenization: Tokenization is the process of breaking a stream of text up into words, phrases, symbols, or other meaningful elements called tokens. The list of tokens becomes input for further processing such as parsing or text mining.
- Stop Words: In computing, stop words are words which are filtered out prior to, or after, processing of natural language data (text). In terms of linguistics these words are called as function words. Words like ’a’, ’an’, ’the’ are examples for stop words. There is no defined set of stop words available. Different applications and research groups uses different sets o stop words.
Generally stop words are omitted in text mining process. The frequency of stop words will be very high in any corpus compared to content words.Pointers to some good stop word list is available at http://en.wikipedia.org/wiki/Stop_words
- Bag of Words: The bag-of-words model is a simplifying representation used in natural language processing and information retrieval (IR). In this model, a text (such as a sentence or a document) is represented as an un-ordered collection of words, disregarding grammar and even word order. Analyzing text by only analyzing frequency of words is called as bag of words model.
- TF-IDF: Tf–idf, term frequency–inverse document frequency, is a numerical statistic which reflects how important a word is to a document in a collection or corpus.
![image1](images/tfidf1.png)
![image2](images/tfidf2.png)
- LDA: In natural language processing, Latent Dirichlet allocation (LDA) is a generative model that allows sets of observations to be explained by unobserved groups that explain why some parts of the data are similar. For example, if observations are words collected into documents, it posits that each document is a mixture of a small number of topics and that each word's creation is attributable to one of the document's topics.

Having set these basic fundamentals, let's try and analyze the problem at hand.

###Algorithm
####Pre-processing
First, we have to process the raw data files. I had two options. First option was to read the XML annonated files and get the data from there. And second, I can read the raw text files. I prefered the second option for simplicity. I could not figure out the advantage of using the XML files over .txt files. But there was an computation overhead for option 1 as compared to option 2. I am skeptical that I have missed out some important feature by not using the XML files. But that's a work for another day and we can improve on the existing work later on. 

So now we have to recursively go into each file and read it's content. This can be done by the code below:


In [ ]:
#
# This is for logging data
#
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger('text_mining_logger')
#
# We will be appending all our documents to this array.
#
documents = []

#
# get the class-names from the directory structure
#
directory_names = list(set(glob.glob(os.path.join("docs", "*"))
                           ).difference(set(glob.glob(os.path.join("docs", "*.*")))))
#
# List of string of class names
#
namesClasses = list()

#
# Navigate through the list of directories recursively
#
logger.info("Reading the documents")
for folder in directory_names:
    #
    # Append the string class name for each class
    #
    currentClass = folder.split(os.pathsep)[-1]
    namesClasses.append(currentClass)

    for fileNameDir in os.walk(folder):
        for fileName in fileNameDir[2]:
            #
            # Only read in the text files
            #
            if fileName[-4:] != ".txt":
                continue
            nameFileImage = "{0}{1}{2}".format(fileNameDir[0], os.sep, fileName)
            with open(nameFileImage, 'r') as myfile:
                #
                # Read the file and remove the new-line characters.
                #
                data=myfile.read().replace('\n', '')
                #
                # Remove all special characters.
                #
                new_string = re.sub('[^a-zA-Z0-9]', ' ', data)
            documents.append(new_string)

As you can see, I have also done some basic filtering. I have removed all special characters from the document. When I was browsing through the files, I noticed that the files contained a lot of unicode special characters and these characters don't have any special significance towards the content. Hence, I filtered these out with a simple regEX.

The next step is to tokenize the words. And we also have to remove the commonly occuring words as "a, an, is, are, will" etc. I am also removing words which appear only once. I found this logic very effective in filtering out various names, places, typos and other anamolies. This is done by the following code:

In [ ]:
#
# Remove common words and tokenize
#
logger.info("Pre-processing the documents to remove common words.")
stoplist = set('for a an of the and to in is are when what how where ha etc et all'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
      for document in documents]
#
# remove words that appear only once
#
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
 for token in text:
     frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
      for text in texts]

###TF-IDF
In this step, I experimented a lot. 
Initally, my approach was to create a bag of words (bow) and then appl LDA on the bad words to generate a big database of topics. Add with each topic I associated top 1000 terms. And based on these terms, I queried the user input terms and determined which topic our word belongs to. After that, it was a simple case of retreiving the other words recursively.
But I found a major flaw in this approach. Topic modelling might not be very suitable in the context of finding out co-occuring words. Topic modelling also limits my application to a hadcoded numer of topics. If, in the future the number of documents decrease or increase significantly, my chosen number of topics might not be the correct number of topics. And even in the present context, I was not able to determine a good number of topics.
Hence, then I shifted to TF-IDF. Now as explained earlier, TF-IDF is basically a matrix. It has the rows as the words and the columns as the documents. The tf-idf weight determines how relevant is the word in that paticular document. For example let's look at the following example:
![image3](images/tfidf3.png)

So, if a word occurs more frequently in a document it has a higher weightage and if it does not occur at all it has 0.
So, I calculated the tf-idf and store the values in a Matrix file. This is done in order to save computation time. If the raw dataset does not change, we need not calculate the tf-idf again. Another important point to note is dealing with string is very precess and time intensive. Hence, I converted all the strings to a number/id and maintained a dictionary which maps the words and ids. This is done in the following code:


In [ ]:
#
# We create a dictionary so that we don't have to deal with strings.
# Working with strings is very expensive. Instead we will be representing each word
# by a id and we will maintaining the mapping of the word <-> id in our dictionary
#
dictionary = corpora.Dictionary(texts)

#
# store the dictionary, for future reference
#
logger.info("Saving Dictionary.")
dictionary.save('files/pubMed-dictionary.dict')

#
# print(dictionary.token2id)
#
id2word = dictionary.token2id

corpus = [dictionary.doc2bow(text) for text in texts]

#
# store to disk, for later use
#
logger.info("Saving the corpus.")
corpora.MmCorpus.serialize('files/pubMed-corpus.mm', corpus)

mm = MmCorpus('files/pubMed-corpus.mm')
logger.info("Computing the TFIDF Matrix.")
tfidf = TfidfModel(mm, id2word=id2word, dictionary=dictionary, normalize=True)

logger.info("Saving the TFIDF.")
MmCorpus.serialize('files/pubMed-tfidf.mm', tfidf[mm], progress_cnt=10000)


Now, I turn to generate the relevant words and create a json file which can be parsed by the visualization application to create a web-interface.
For this I queried the user input terms and determined which document our word belongs to. After that, I retrieved all other words and calculated their cumulative weightage. Based on the final cumulative weightage, I picked the top 5 most relevant words. This process was repeated recusively for the number of layers the user has specified. Finally, I saved the json file. The code i as follows:

Loading the Dictionary File and the TF-IDF file

In [ ]:
__author__ = 'mrtyormaa'

from nltk.compat import raw_input
from scipy.io import mmread
import operator
import json
from gensim import corpora
import logging

#
# This is for logging data
#
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger('text_mining_logger')

# Pre-processing steps required to load the Dictionary and TFIDF
# Retrieve the dictionary
#
logger.info("Loading the dictionary.")
dictionary = corpora.Dictionary.load('files/pubMed-dictionary.dict')

#
# Let's tokenize the dictionary and get the word <-> id pairs.
#
id2word = dictionary.token2id

#
# inverse dictionary. This is easy for future lookups of ids
#
word2id = {val: key for key, val in id2word.items()}

#
# Retrieve the TFIDF
#
logger.info("Loading the TFIDF.")
file = mmread('files/pubMed-tfidf.mm')

Helper functions to get the relevant documents and the co-occuring words

In [ ]:
#
# This method returns all the documents containing the given word.
#
def getDocumentsWithParam(param):
    docs = []
    id = id2word[param]
    i = 0
    for col in file.col:
        if col == id:
            docs.append(file.row[i])
        i += 1
    return docs


#
# This method returns all the words in the document.
#
def getWordsAndTFIDF(docId):
    data = []
    i = 0
    for row in file.row:
        if docId == row:
            data.append([file.col[i], file.data[i]])
        i += 1
    return data


User Input for the WORD and the number of levels:

In [ ]:
#
# User input
#
print("Enter a word for which you want co-occurance: ")
user_word = raw_input()
print("How many layers of output do you want to generate(0-5): ")
level = int(raw_input())

Finally, the recursive algorithm to generate the entire heirarchial tree and the JSON file

In [ ]:

#
# This is a recursive method used to generate the relevant words
#
def generateRelevantWords(param, l):
    docs = getDocumentsWithParam(param)
    relevant_words = {}
    for doc in docs:
        data = getWordsAndTFIDF(doc)
        for d in data:
            #
            # Ignore already visited words. This is to avoid repetition of words.
            #
            if word2id[d[0]] in visited_words:
                continue
            if not d[0] in relevant_words:
                relevant_words[d[0]] = 0.0
            relevant_words[d[0]] += d[1]
    #
    # One of the base cases of recursion. If we don't find relevant words, return null array.
    #
    if not relevant_words:
        return []
    #
    # We sort the words in decreasing order of their TFIDF values. This gives us tuples of
    # word and tfidf. (word, tfidf)
    #
    word_tfidf_tuples = sorted(relevant_words.items(), key=operator.itemgetter(1), reverse=True)

    #
    # Let's limit the relevant words to 5 for each word.
    #
    word_tfidf_tuples = word_tfidf_tuples[0:5]

    for word in word_tfidf_tuples:
        visited_words.append(word2id[word[0]])
    tree = []

    for word in word_tfidf_tuples:
        current_word = []
        current_word.append(word2id[word[0]])
        topic = {}
        topic['words'] = current_word
        topic['name'] = 'Topic_0'
        if (l == level - 1):
            tree.append(topic)
        else:
            #
            # Go another deeper level.
            #
            result = generateRelevantWords(word2id[word[0]], l + 1)
            if result:
                topic['children'] = result
            tree.append(topic)
    return tree


json_response = {}
visited_words = []
visited_words.append(user_word)
json_response['name'] = user_word
json_response['words'] = [user_word]

logger.info("Generating the Relevant Words.")
json_response['children'] = generateRelevantWords(user_word, 0)

logger.info("Saving the JSON file.")
with open('/Users/mrtyormaa/Sites/Hierarchie/app/data/pub-med.json', 'w') as outfile:
    json.dump(json_response, outfile)

###Visualization
I am using the Heirarchy visualization librabry as for visualization. It generates a very nice sunburst image. And the sunburst can b navigated deeper on different levels. 
![viz1](images/visualization.png)
![viz2](images/visualization1.png)
Very few code changes have been made from the original source. In fact, it is a very simple AngularJS application which uses Decisive Analyitc's librabry for generating the sunburst. I have editted the view models and the controllers to tailor the visualization according to my need. 
A live demo of the project can be seen [here](http://asutosh-satapathy.com/#/) which creates Sunburst for two words "Ebola" and "Experiments"

##Reflections
1. Understanding the fundamentals or getting the domain knowledge is most crucial. I did not have any experince with text mining before. Hence, I spent a significant amount of time understanding the fundamentals. Once, That was out of the way implementation boiled down a lot experiments and judging the results.
2. As I have not used Map Reduce, the application is really slow. It takes apprximately 9 hours to calulate the tf-idf and then takes another 9 hours to calulcate the relevant words. 
3. The computing time can be significantly reduced by MapReduce. Hence, the immediate next step of this project will be to implement the same.
4. When I generated the sunburst with 10 words per each word, the sunburst was huge and there was a lot of information. It was very difficult to navigate through all the information. Hence, i experimented with a few different numbers and finally fixed the size to 4.
5. As the number of words is exponential in growth, increasing each level creates a signifiant difference in computing time. We are also doing recursion to generate all relevant words which makes things even slower.
6. In my opinion this can be implemented in daily life. For example news. We can create sunburst for for dailynews and get the key points of the daily day to day news.